In [1]:
# import pandas

# iris_df = pandas.read_csv('https://raw.githubusercontent.com/' +
#                           'pydata/pandas/master/pandas/tests/' + 
#                           'data/iris.csv')

# from sklearn.tree import DecisionTreeClassifier
# from sklearn2pmml.pipeline import PMMLPipeline

# pipeline = PMMLPipeline([
# 	("classifier", DecisionTreeClassifier())
# ])
# pipeline.fit(iris_df[iris_df.columns.difference(["Name"])], iris_df["Name"])

# from sklearn2pmml import sklearn2pmml

# sklearn2pmml(pipeline, "DecisionTreeIris.pmml", with_repr = True)

In [32]:
import warnings
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [33]:
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression

from sasctl import Session, register_model, publish_model


# Load the Iris data set and convert into a Pandas data frame.
raw = datasets.load_iris()
X = pd.DataFrame(raw.data, columns=['SepalLength', 'SepalWidth',
                                    'PetalLength', 'PetalWidth'])
y = pd.DataFrame(raw.target, columns=['Species'], dtype='category')
y.Species.cat.categories = raw.target_names

# Fit a sci-kit learn model
model = LogisticRegression()
model.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

Move the model to Model Manager

In [34]:
import swat
import _config # this is a local config file that has the credentials to authenticate to CAS

In [4]:
host = _config.race_login()[2]
user = _config.race_login()[0]
passwd = _config.race_login()[1]

In [5]:
host,user,passwd

('pdcesx17022.exnet.sas.com', 'sasdemo', 'Orion123')

In [6]:
!pip show sasctl

Name: sasctl
Version: 1.2.3
Summary: SAS Viya REST Client
Home-page: https://github.com/sassoftware/python-sasctl/
Author: SAS
Author-email: None
License: Apache v2.0
Location: /opt/anaconda/anaconda3/envs/dl4viya/lib/python3.6/site-packages
Requires: six, requests
Required-by: 


In [7]:
with Session(hostname=host, 
             username=user, 
             password=passwd, 
             verify_ssl=False):
    model_name = 'iris3'
    register_model(model,
                   model_name,
                   input=X,         # Use X to determine model inputs
                   project='iris3',  # Register in "Iris" project
                   force=True)      # Create project if it doesn't exist

    # Publish the model to the real-time scoring engine
    module = publish_model(model_name, 'maslocal')

    # Select the first row of training data
    x = X.iloc[0, :]

    # Call the published module and score the record
    result = module.score(**x)
    print(result)

/opt/anaconda/anaconda3/envs/dl4viya/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/anaconda/anaconda3/envs/dl4viya/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


setosa


In [8]:
import sasctl

conn = sasctl.Session(hostname=host, 
             username=user, 
             password=passwd, 
             verify_ssl=False )

In [16]:
project = sasctl.mr.get_project('iris3')

/opt/anaconda/anaconda3/envs/dl4viya/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [18]:
 #microanalytic_score service has get_module().  
    #The returned object should automatically have functions defined that correspond to the module steps

In [19]:
module=sasctl.services.microanalytic_score.get_module('iris3')

/opt/anaconda/anaconda3/envs/dl4viya/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [30]:
sasctl.services.microanalytic_score.execute_module_step('iris3','score',**dict(x))

OrderedDict([('var1', 'setosa'), ('rc', 0), ('msg', None)])